1. Import libs and read excel (reading excel is slow)

In [1]:
from pprint import pprint
import pandas as pd
import re
data = pd.read_excel('../../Clarifications/G4 - mandatory data elements 11.10.2023 (1) - Copy.xlsx')


C:\Users\ciar\AppData\Local\Temp\ipykernel_16260\2590737069.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


2. Extract elements and sub-element names from clarifications excel sheet

In [59]:
print(data.keys())
clarification_elements = {}


'''Search by name:'''
for i in range(data.shape[0]):
    element = data.at[i, 'Data element/class name ']
    if element != element:
        element = data.at[i, 'Data sub-element/sub-class name']
    clarification_elements[element] = element = data.at[i, 'DE No. (Ref ID6)']

    
print(clarification_elements)


Index(['DE No. (Ref ID6)', 'Data element/class name ',
       'Data sub-element/sub-class name', 'Ref ID5', 'G4',
       'Should the data element (from column B) be filled out on the declaration? ',
       'If the data element (from column B) is being declared on the declaration - which data sub-elements (from column C) should then ALWAYS also be filled out? ',
       'Initial version', 'Comments'],
      dtype='object')
{'Goods item number': '11 03 001 000', 'Specific circumstance indicator': '11 04 001 000', 'Previous document': '12 01 000 000', 'Reference number': '17 10 001 000', 'Type': '13 13 029 002', 'Goods item identifier': '12 01 007 000', 'Additional information': '12 02 000 000', 'Code': '12 02 008 000', 'Text': '12 02 009 000', 'Supporting document': '12 03 000 000', 'Additional reference': '12 04 000 000', 'Transport document': '12 05 000 000', 'Reference number/UCR': '12 08 001 000', 'LRN': '12 09 001 000', 'Warehouse': '12 11 000 000', 'Identifier': '19 10 015 000', 'Co

3. Extract EUCDM name annotations from XSD file
(NOTE: this assumes all elements are correctly annotated, which for G4 they are because I manually edited them)

In [94]:

clarifications_filename = './G4_DMS_v0.1.xsd'

def extract_elements(text, search_by):
    # Define the regex pattern
    if search_by == 'Name':
        pattern = r"<EUCDMname>(.*?)</EUCDMname>"
    elif search_by == 'Number':
        pattern = r"<code>(.*?)</code>"
    elif search_by == 'Both':
        pattern = (
                r"<EUCDMname>(.*?)</EUCDMname>\s*"
                r"<EUCDMversion>(.*?)</EUCDMversion>\s*"
                r"<code>(.*?)</code>"
            )


    # Use re.findall to extract all occurrences
    matches = re.findall(pattern, text)
    # Return the matches
    return matches

text = open(clarifications_filename, 'r').read()
# element_name_annotations = extract_elements(text, 'Name')
# element_number_annotations = extract_elements(text, 'Number')
# print(len(element_number_annotations), len(element_name_annotations))
annotations = extract_elements(text, 'Both')
print(annotations)
#xxx


#schema_annotations = {element_name_annotations[i]: element_number_annotations[i] for i in range(len(element_name_annotations))} 
schema_annotations = {annotations[i][0]: annotations[i][2] for i in range(len(annotations))} 

print(schema_annotations)
if '19 10 015 000' not in schema_annotations.values():
    print("No")


[('LRN', 'EUCDM 6.1', '12 09 001 000'), ('Specific circumstance indicator', 'EUCDM 6.1', '11 04 001 000'), ('Representative', 'EUCDM 6.1', '13 06 000 000'), ('Name', 'EUCDM 6.1', '13 06 016 000'), ('Identification number', 'EUCDM 6.1', '13 06 017 000'), ('Address', 'EUCDM 6.1', '13 03 018 000'), ('City', 'EUCDM 6.1', '13 03 018 022'), ('Country', 'EUCDM 6.1', '13 03 018 020'), ('Street additional line', 'EUCDM 6.1', '13 06 018 024'), ('Postcode', 'EUCDM 6.1', '13 06 018 021'), ('Street', 'EUCDM 6.1', '13 06 018 023'), ('Number', 'EUCDM 6.1', '13 06 018 025'), ('P.O. box', 'EUCDM 6.1', '13 06 018 026'), ('Sub-division', 'EUCDM 6.1', '13 06 018 027'), ('Communication', 'EUCDM 6.1', '13 06 029 000'), ('Identifier', 'EUCDM 6.1', '13 06 029 015'), ('Type', 'EUCDM 6.1', '13 06 029 002'), ('Status', 'EUCDM 6.1', '13 06 030 000'), ('Declarant', 'EUCDM 6.1', '13 05 000 000'), ('Name', 'EUCDM 6.1', '13 05 016 000'), ('Identification number', 'EUCDM 6.1', '13 05 017 000'), ('Address', 'EUCDM 6.1'

TypeError: unhashable type: 'list'

In [87]:
schema_annotations =  {k.lower(): v for k, v in schema_annotations.items()}
print("The following elements from the clarifications excel sheet are missing from the XSD: ")
for key, val in clarification_elements.items():
    if key.lower() not in schema_annotations.keys(): #or val not in schema_annotations.values():
        print(val + " " + key)
print("\n")
print(schema_annotations.values())

for key, val in schema_annotations.items():
    if val == '12 01 000 000':
        print("Yes")


for key, val in clarification_elements.items():
    if val not in schema_annotations.values():
        print(val + " " + key)

The following elements from the clarifications excel sheet are missing from the XSD: 
16 15 048 000 GNSS
16 15 081 000 Postcode address
17 10 000 000 Supervising customs office
18 09 056 000 Harmonized System sub-heading code


dict_values(['12 09 001 000', '11 04 001 000', '13 06 000 000', '13 03 016 000', '13 14 017 000', '13 13 018 000', '13 13 018 022', '16 14 020 000', '13 13 018 024', '13 13 018 021', '13 13 018 023', '13 13 018 025', '13 13 018 026', '13 13 018 027', '13 13 029 000', '12 11 015 000', '12 05 002 000', '13 06 030 000', '13 05 000 000', '18 03 001 000', '19 11 001 000', '15 04 001 000', '12 02 000 000', '12 02 008 000', '12 02 009 000', '12 04 000 000', '12 05 001 000', '13 14 000 000', '13 14 031 000', '19 06 000 000', '19 06 061 000', '13 03 000 000', '13 13 028 000', '11 03 001 000', '18 06 000 000', '18 06 054 000', '18 06 004 000', '18 06 003 000', '18 09 000 000', '18 05 001 000', '18 08 000 000', '18 09 056 000', '18 09 057 000', '18 04 001 000', '12 01 000 